# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:

import numpy as np
import os
print (os.getcwd())
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from PIL import Image

if tf.__version__ != '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.0!')

# This is needed to display the images.
%matplotlib inline



/Users/Erik/git/paintspot


/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [2]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet101_coco_2017_11_08'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'pretrained_model/%s/frozen_inference_graph.pb' % MODEL_NAME


## Download Model

In [3]:
if not os.path.exists('pretrained_model/%s' % MODEL_NAME):
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Helper code

In [5]:
def load_image_into_numpy_array(image):
    im_width, im_height = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
from watson_developer_cloud import VisualRecognitionV3

WATSON_API_KEY = '05572b8833ce3e4086391ffa8c55b50381ec6d15'

# Replace with your api key
visual_recognition = VisualRecognitionV3('2017-11-30', api_key=WATSON_API_KEY)

MAX_NUMBER_OF_BOXES = 10
MINIMUM_CONFIDENCE = 0.6

COLORS = ['b', 'g', 'r', 'c', 'm', 'y', 'b', 'w']

# Detection

In [ ]:
import scipy.misc
import json

images = ["mauritshuis"]

img_height, img_width = None, None
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        for image_name in images:
            print ("Opening file: 'images/%s.jpg'" % image_name)
            image = Image.open('images/%s.jpg' % image_name)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            
            img_height, img_width, _ = image_np.shape
            
            # Create figure and axes and display the image
            fig = plt.figure(figsize=(24, 16), frameon=False)
            fig = plt.imshow(image_np)
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            print ("+ Detecting objects")
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            boxes = np.squeeze(boxes)
            classes = np.squeeze(classes).astype(np.int32)
            scores = np.squeeze(scores)

            # From normalized to pixel coordinates
            boxes[:, [0, 2]] *= img_height
            boxes[:, [1, 3]] *= img_width
            
            print ("+ Classifying objects")
            for box, score, label in zip(boxes, scores, classes):
                
                by1, bx1, by2, bx2 = box.astype(int)
                
                box_height, box_width = (by2-by1), (bx2-bx1)
                
                if box_width > 0.4*img_width or score < 0.1:
                    continue
                
                # Save cropped image
                obj = image.crop((bx1, by1, bx2, by2))
                obj.save('tmp.jpg')
                
                # Open cropped image and classify using Watson
                with open('tmp.jpg', 'rb') as image_file:
                    watson_params = json.dumps({'threshold': 0.6, 'classifier_ids': ['default']})
                    results = visual_recognition.classify(images_file=image_file, parameters=watson_params)
                    label = results['images'][0]['classifiers'][0]['classes'][0]['class']
                    ax = plt.gca()
                    ax.text(bx1 + 5, 
                            by1 - 5, 
                            label, 
                            fontsize=10, 
                            color='white', 
                            bbox={'facecolor': 'g', 'edgecolor':'none'})

                # Create a Rectangle patch
                rect = patches.Rectangle((bx1, by1), 
                                         box_width, 
                                         box_height, 
                                         linewidth=2, 
                                         edgecolor='g', 
                                         facecolor='none')
                ax = plt.gca()
                ax.add_patch(rect)
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False)
            print ("Writing image to 'images/output_%s.jpg'" % image_name)
            plt.savefig('images/output_%s.jpg' % image_name, bbox_inches='tight', pad_inches=0.0)
            plt.show()
            plt.close()
            print ()
        

Opening file: 'images/mauritshuis.jpg'
+ Detecting objects
+ Classifying objects
